## A. Downloading and testing different VGG weights

In [1]:
import numpy as np
from vgg16 import VGG16
# from resnet50 import ResNet50
from keras.preprocessing import image
from imagenet_utils import preprocess_input, decode_predictions

Using TensorFlow backend.


#### Loading Full Vgg16 model and testing on a sample image

In [7]:
model = VGG16(include_top=True, weights='imagenet')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Testing VGG16 Model on an image of a Snow Owl

In [13]:
img_path = 'owl.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)
preds = model.predict(x)
print('Predicted:', decode_predictions(preds))

('Input image shape:', (1, 224, 224, 3))
('Predicted:', [[(u'n01614925', u'bald_eagle', 0.62520725), (u'n01608432', u'kite', 0.34001073), (u'n01622779', u'great_grey_owl', 0.031670108), (u'n01819313', u'sulphur-crested_cockatoo', 0.0016022217), (u'n02058221', u'albatross', 0.00076713879)]])


Checking out the last layer

In [16]:
model.layers[-1].get_config()

{'activation': 'softmax',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'predictions',
 'trainable': True,
 'units': 1000,
 'use_bias': True}

#### Loading Vgg16 model without the fully connected layers and testing on a sample image ( Basically this network will act as basic feature extractor)

In [27]:
model = VGG16(weights='imagenet', include_top=False)
model.summary()
model.layers[-1].get_config()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

{'data_format': 'channels_last',
 'name': 'block5_pool',
 'padding': 'valid',
 'pool_size': (2, 2),
 'strides': (2, 2),
 'trainable': True}

In [49]:
img_path = 'owl.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

#### The shape of the feature will be same as the last pool layer in block5
#### block5_pool (MaxPooling2D)   (None, 7, 7, 512)         0  

print features.shape 

(1, 7, 7, 512)


## B. Transfer Learning on a Custom dataset

Our custom data contains four image folder:<br/>
1. cat<br/>
2. dog<br/>
3. horse<br/>
4. human<br/><br/>

We are going to change the last layer of the model (classification layer).
Original vgg model is built for 1000 class. We are going to edit it and tweak the model for 4 class.

In [1]:
import numpy as np
import os
import time
from vgg16 import VGG16
from keras.preprocessing import image
from imagenet_utils import preprocess_input, decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
img_path = 'owl.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
print (x.shape)
x = np.expand_dims(x, axis=0)
print (x.shape)
x = preprocess_input(x)
print('Input image shape:', x.shape)

(224, 224, 3)
(1, 224, 224, 3)
('Input image shape:', (1, 224, 224, 3))


In [6]:
PATH = os.getcwd() # get current directory
data_path = PATH + '/data' #currentdirectory+/data -where data is the dataset folder 
data_dir_list = os.listdir(data_path)
print data_dir_list # will print out 4 class

['Humans', 'dogs', 'horses', 'cats']


In [15]:
img_data_list=[] # list which will contain the preprocessed image

# preprocessing images
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        img_path = data_path + '/'+ dataset + '/'+ img 
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
#        x = x/255
#        print('Input image shape:', x.shape)
        img_data_list.append(x)
    

# Preparing the image dataset based on vgg input dimensions
img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)


Loaded the images of dataset-Humans

Loaded the images of dataset-dogs

Loaded the images of dataset-horses

Loaded the images of dataset-cats

(808, 1, 224, 224, 3)
(1, 808, 224, 224, 3)
(808, 224, 224, 3)


In [19]:
# Define the number of classes
num_classes = 4
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64') # creating labels

# encoding labels based on number of classes 
labels[0:202]=0
labels[202:404]=1
labels[404:606]=2
labels[606:]=3

names = ['cats','dogs','horses','humans']

# # convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

# #Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

#### Loading Full Vgg16 model and performing only TL (training only the last fully connected classfication layer)

In [43]:
# Custom_vgg_model_1
#Training the classifier alone
image_input = Input(shape=(224, 224, 3))

#loading the model
model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

# getting upto the second last layer and dropping off the last classification layer
last_layer = model.get_layer('fc2').output 

#creating a new classification layer based on our num_classes and adding it
# to the last_layer (second last layer of full vgg16 model)

out = Dense(num_classes, activation='softmax', name='output')(last_layer)

#creating a new custom model 
custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [44]:
print "Is layer 3 trainable before freezing:"+str(custom_vgg_model.layers[3].trainable)

#Since we will only have to train our new classifier (last layer) so we
#will freeze all the other layers except the last layer

for layer in custom_vgg_model.layers[:-1]: 
    layer.trainable = False
    
print "Is layer 3 trainable after freezing:"+str(custom_vgg_model.layers[3].trainable)

Is layer 3 trainable before freezing:True
Is layer 3 trainable after freezing:False


In [46]:
# compiling our custom vgg16 model

# custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

# training our custom vgg16 model
hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=12, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % (t - time.time()))
# (loss, accuracy) = custom_vgg_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

# print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))


Train on 646 samples, validate on 162 samples
Epoch 1/12


KeyboardInterrupt: 